## This code will iterate over multiple Surface Reflectance PlanetScope images:
<ol>

  <li>Calculate NDVI and associated stats (min, max, mean, etc.) </li>
  <li>Plot indivudial NDVI raster for each image</li>
  <li>Plot average NDVI temporal curve</li>
  <li>Optional: Export NDVI raster or plots into PNGs</li>

</ol>

In [ ]:
import os
from osgeo import gdal
import subprocess
import numpy as np
import rasterio
import matplotlib.dates as mdates
import matplotlib.pylab as plt
from pandas.plotting import register_matplotlib_converters
from datetime import datetime
from rasterio.plot import show
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import folium
import json
from geojson_utils import centroid

### File Locations

### Note about directory format:

* Windows directory format for **files_location** below: `"C:\\subdirectory\\directory\\"`
* Mac Directory format for **files_location** below: `"/Users/username/directory/"`

In [ ]:
aoi_fpath = r"/your/GeoJSON/directory/aoi.geojson" #GeoJSON location

basemap_name = "global_monthly_2019_04_mosaic" # Planet Mosaic Name (Optional)

files_location = "/your/PlanetScope/directory/" #Downloaded PlanetScope Surface Reflectance images


### Additional Options

In [ ]:
#Set to True or False

write_NDVI_raster = False # Generates NDVI raster for each Surface Reflectance image
write_NDVI_plot_figs = True # Generates individual PNGs for each NDVI plot
write_NDVI_temporal_plot = False # Generates a PNG of the NDVI temporal plor

### Create Map (Optional)

In [ ]:
api_key = os.getenv('PL_API_KEY') # Load API Key by Environment variable

#Read GeoJSON
with open(aoi_fpath) as f:
    geom = json.load(f)

geom_centroid = centroid((geom["features"][0]["geometry"]))["coordinates"]
geom_centroid.reverse()
    
#Define Planet's Basemap
planet_basemap = "https://tiles0.planet.com/basemaps/v1/planet-tiles/%s/gmap/{z}/{x}/{y}.png?api_key=%s" % (basemap_name, api_key)

#Add Planet's XYZ Monhly Mosaic Basemap as a permanent baselayer
map = folium.Map(location=geom_centroid, zoom_start=12, tiles=None)

folium.GeoJson(geom, name='GeoJSON').add_to(map)

folium.raster_layers.TileLayer(
    tiles=planet_basemap,
    attr='Planet Inc.',
    name=basemap_name,
    max_zoom=16,
    overlay=False,
    control=True,
).add_to(map)
map

map.add_child(folium.LayerControl())

In [ ]:
image_files = [image for image in os.listdir(files_location) if image.endswith('.tif') if not ('NDVI' in image or 'NDWI' in image or 'ndvi' in image or 'ndwi' in image or 'udm' in image or 'visual' in image)]
image_files.sort()
image_files_num = len(image_files)

print(f"There {image_files_num} TIF files in directory {files_location}\n")

for x in range(image_files_num):
    print(image_files[x])

In [ ]:
ndvi_avgs = {}
for filename in image_files:
    infilename = files_location + filename
    print(infilename)
    with rasterio.open(infilename) as src:
        red = src.read(3)
    with rasterio.open(infilename) as src:
        nir = src.read(4)

    # Change the data type from uint8 to float so that we can have floating point numbers stored in our arrays
    red = red.astype(float)
    nir = nir.astype(float)
    # Allow 0 division in numpy
    np.seterr(divide='ignore', invalid='ignore')
    # Calculate NDVI
    ndvi = (nir - red)/(nir + red)
    # check range NDVI values, excluding NaN
    print(f'NDVI MIN:{np.nanmin(ndvi)}\nNDVI MAX:{np.nanmax(ndvi)}\nNDVI MEAN:{np.nanmean(ndvi)}')
    #show(ndvi, cmap='summer')
    class MidpointNormalize(colors.Normalize):
        """
        Normalise the colorbar so that diverging bars work there way either side from a prescribed midpoint value)
        e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-100, vmax=100))
        Credit: Joe Kington, http://chris35wills.github.io/matplotlib_diverging_colorbar/
        """
        def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
            self.midpoint = midpoint
            colors.Normalize.__init__(self, vmin, vmax, clip)

        def __call__(self, value, clip=None):
            # I'm ignoring masked values and all kinds of edge cases to make a
            # simple example...
            x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
            return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

    # Set min/max values from NDVI range for image (excluding NAN)
    # set midpoint according to how NDVI is interpreted: https://earthobservatory.nasa.gov/Features/MeasuringVegetation/
    min=np.nanmin(ndvi)
    max=np.nanmax(ndvi)
    mid=0.1

    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111)

    # diverging color scheme chosen from https://matplotlib.org/users/colormaps.html
    cmap = plt.cm.RdYlGn 

    cax = ax.imshow(ndvi, cmap=cmap, clim=(min, max), norm=MidpointNormalize(midpoint=mid,vmin=min, vmax=max))

    ax.axis('off')
    ax.set_title('Normalized Difference Vegetation Index', fontsize=18, fontweight='bold')
    plt.suptitle(f'{filename}')

    cbar = fig.colorbar(cax, orientation='horizontal', shrink=0.65)
    
    if write_NDVI_plot_figs:
        fig.savefig(f'{filename}_ndvi_fig.png', dpi=200, bbox_inches='tight', pad_inches=0.7)
    
    plt.show()
    
    if write_NDVI_raster:
        # Set spatial characteristics of the output object to mirror the input
        kwargs = src.meta
        kwargs.update(dtype=rasterio.float32,count = 1)
        # Write band calculations to a new raster file
        with rasterio.open(f'{infilename}_NDVI.tif', 'w', **kwargs) as dst:
            dst.write_band(1, ndvi.astype(rasterio.float32))
    
    date = os.path.basename(infilename).split('_')[0]
    date_obj = datetime.strptime(date, "%Y%m%d")
    band_avg = np.nanmean(ndvi)
    ndvi_avgs[date_obj] = band_avg
    

In [ ]:
ndvi_tuple = sorted(ndvi_avgs.items())
x_ndvi, y_ndvi = zip(*ndvi_tuple)

plt.figure(1, figsize=(20, 10))
plt.plot_date(x_ndvi, y_ndvi, 'b-', tz=None, xdate=True, ydate=False, color='blue', label='NDVI')
plt.plot_date(x_ndvi, y_ndvi, 'g^', tz=None, xdate=True, ydate=False, color='green', label='NDVI Points')
plt.legend(loc='upper right')
plt.xlabel("Date")
plt.ylabel("Avg NDVI")
if write_NDVI_temporal_plot:
    plt.savefig(f'{files_location}ndvi_temporal_plot.png')
plt.show()
